In [1]:
import sys
import numpy as np
import pandas as pd
import scipy
import sklearn
import seaborn
import matplotlib
import tensorflow as tf
import tensorflow_text as tf_text
import keras

# Check versions
print("Python version:", sys.version)
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("SciPy version:", scipy.__version__)
print("scikit-learn version:", sklearn.__version__)
print("Seaborn version:", seaborn.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)
print("TensorFlow Text version:", tf_text.__version__)



Python version: 3.10.15 | packaged by conda-forge | (main, Oct 16 2024, 01:15:49) [MSC v.1941 64 bit (AMD64)]
NumPy version: 1.26.4
Pandas version: 2.2.2
SciPy version: 1.13.1
scikit-learn version: 1.4.2
Seaborn version: 0.13.2
Matplotlib version: 3.9.2
TensorFlow version: 2.10.0
Keras version: 2.10.0
TensorFlow Text version: 2.10.0


In [37]:
#importing the neccessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
import tokenization
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, LSTM,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import roc_auc_score,f1_score

In [33]:
import zipfile
import os
import subprocess

# Step 1: Define the path to your zip file
zip_file_path = "Tokenization.py.zip"

# Step 2: Define the extraction directory
extraction_dir = "extracted_files"

# Step 3: Create the extraction directory if it doesn't exist
if not os.path.exists(extraction_dir):
    os.makedirs(extraction_dir)

# Step 4: Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

print(f"Files have been extracted to: {extraction_dir}")

# Step 5: List the extracted files
print("Extracted files:")
print(os.listdir(extraction_dir))

# Step 6: Construct the full path to the Tokenization.py file
file_path = os.path.join(extraction_dir, "Tokenization.py")

# Step 7: Use subprocess to run the Python script and capture errors
try:
    result = subprocess.run(["python", file_path], capture_output=True, text=True, check=True)
    print("Script ran successfully!")
    print("Output:\n", result.stdout)
except subprocess.CalledProcessError as e:
    print(f"Error while running the script: {e}")
    print("Standard Output:\n", e.stdout)
    print("Error Output:\n", e.stderr)


Files have been extracted to: extracted_files
Extracted files:
['Tokenization.py']
Error while running the script: Command '['python', 'extracted_files\\Tokenization.py']' returned non-zero exit status 1.
Standard Output:
 
Error Output:
 C:\Users\ritid\anaconda3\envs\tf_env_3.10\python.exe: can't find '__main__' module in 'C:\\Users\\ritid\\extracted_files\\Tokenization.py'



In [39]:
data = pd.read_csv('feature_extracted_1.csv')

In [51]:
#will take out only query and label columns only
data = data[['Query','Label']]

In [53]:
data.head()

,Query,Label
0,""" or pg_sleep ( __time__ ) --",1
1,create user name identified by pass123 tempora...,1
2,and 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1


In [55]:
y  = data['Label']
x = data.drop('Label',axis = 1)

In [57]:
#splitting the data into train and test
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,stratify=y)

In [59]:
x_train.shape

(21631, 1)

In [61]:
x_test.shape

(9271, 1)

Will choose maximum sequence length to be 400

<h2> 6.1 Bert Pretrained model loading </h2>

In [41]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text
max_seq_length = 400  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-l-12-h-768-a-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])


In [43]:
from tensorflow.keras.models import Model
#Bert model
#We are using only pooled output not sequence out. 
#If you want to know about those, please read https://www.kaggle.com/questions-and-answers/86510
bert_model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=pooled_output)

In [45]:
bert_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 128)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 128, 768)]                'input_mask[0][0]',       

In [47]:
from tensorflow.python.framework import ops
from bert import tokenization
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer=tokenization.FullTokenizer(vocab_file,do_lower_case )

<h2> 6.2 Tokenization </h2>

In [81]:
from absl import flags

# Manually set the value of the flag before tokenization
flags.FLAGS.preserve_unused_tokens = False

# Now you can call the tokenizer function
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokens(text_input):
    li_tokens = []
    li_tokens_to_ids = []
    li_mask = []
    li_segment = []

    for i in range(len(text_input)):
        token = tokenizer.tokenize(text_input[i])

        if len(token) > max_seq_length - 2:
            token = token[0:(max_seq_length - 2)]
        
        token = ['[CLS]', *token, '[SEP]']
        
        if len(token) < max_seq_length:
            pad_to_add = max_seq_length - len(token)
            for _ in range(pad_to_add):
                token.append('[PAD]')
        
        input_ids = tokenizer.convert_tokens_to_ids(token)
        li_tokens.append(token)
        li_tokens_to_ids.append(input_ids)
        li_mask.append([1 if i > 0 else 0 for i in input_ids])
        li_segment.append([0] * max_seq_length)
    
    return np.array(li_tokens), np.array(li_tokens_to_ids), np.array(li_mask), np.array(li_segment)

# Now you can call the function with your data
X_train_tokens_, X_train_tokens, X_train_mask, X_train_segment = tokens(x_train['Query'].values)
X_test_tokens_, X_test_tokens, X_test_mask, X_test_segment = tokens(x_test['Query'].values)


In [83]:
print(X_train_tokens_[0])

print(X_train_tokens[0])
print(X_train_mask[0])
print(X_train_segment[0])

['[CLS]' '1' '%' '"' ')' ')' '(' 'select' '(' 'case' 'when' '(' '54'
 '##51' '=' '54' '##51' ')' 'then' 'reg' '##ex' '##p' '_' 'sub' '##st'
 '##ring' '(' 'repeat' '(' 'right' '(' 'char' '(' '54' '##51' ')' ',' '0'
 ')' ',' '5000' '##00' '##00' '##0' ')' ',' 'null' ')' 'else' 'char' '('
 '108' ')' '|' '|' 'char' '(' '76' ')' '|' '|' 'char' '(' '112' ')' '|'
 '|' 'char' '(' '116' ')' 'end' ')' 'from' 'information' '_' 'sc' '##hema'
 '.' 'system' '_' 'users' ')' 'and' '(' '(' '"' '%' '"' '=' '"' '[SEP]'
 '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]'
 '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]'
 '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]'
 '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]' '[PAD]']
[  101  1015  1003  1000  1007  1007  1006  7276  1006  2553  2043  1006
  5139 22203  1027  5139 22203  1007  2059 19723 10288  2361  1035  4942
  3367  4892  1006  9377  1006  2157  1006 25869  1006 

In [85]:
#getting embeddings from bert model
bert_model.input

[<KerasTensor: shape=(None, 128) dtype=int32 (created by layer 'input_word_ids')>,
 <KerasTensor: shape=(None, 128) dtype=int32 (created by layer 'input_mask')>,
 <KerasTensor: shape=(None, 128) dtype=int32 (created by layer 'segment_ids')>]

In [87]:
bert_model.output

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'keras_layer_1')>

In [89]:
X_train_pooled_output=bert_model.predict([X_train_tokens,X_train_mask,X_train_segment])
X_test_pooled_output=bert_model.predict([X_test_tokens,X_test_mask,X_test_segment])

290/290 [==============================] - 1553s 5s/step


In [93]:
train_y = tf.keras.utils.to_categorical(y_train)
test_y = tf.keras.utils.to_categorical(y_test)

In [97]:
import os
import datetime
import shutil
import tensorflow as tf

# Define the log directory
log_dir = os.path.join("logs", 'fits', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

# Remove the directory if it exists (cross-platform way)
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)

# Now, create the TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True)

# Example: Assuming you have a model, you can use it like this:
# model.fit(x_train, y_train, epochs=10, callbacks=[tensorboard_callback])


In [99]:
#creating custom callbacks for f1-score metrics
class Metrics(tf.keras.callbacks.Callback):
  def __init__(self,validation_data):
    self.x_test = validation_data[0]
    self.y_test = validation_data[1]

  def on_train_begin(self, logs={}):
        self.history={'loss': [],'accuracy': [],'val_loss': [],'val_accuracy': [],'val_f1': []}
  def on_epoch_end(self,epoch,logs = {}):
    val_predict = (np.asarray(self.model.predict(self.x_test)))
    val_label = np.argmax(val_predict,axis = 1)
    val_target = np.argmax(self.y_test,axis = 1)
    val_f1 = f1_score(val_target,val_label)
    self.history['val_f1'].append(val_f1)
    print("val_f1_score",val_f1)

In [101]:
from tensorflow.keras.callbacks import ModelCheckpoint
import os
tf.keras.backend.clear_session()  # For easy reset of notebook state.

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,mode = "max",
                                                 save_weights_only=True,save_best_only = True,
                                                 verbose=1)

In [103]:
#creating objects for metrics custom callback
score = Metrics((X_test_pooled_output,test_y))

In [105]:
input_layer=Input(shape=(X_train_pooled_output.shape[1],))
dense_0 = Dense(524,activation = "relu")(input_layer)
dense_ = Dense(256,activation="relu")(dense_0)
drop_0 = Dropout(0.5)(dense_)
dense_1 = Dense(128,activation = "relu")(drop_0)
drop_1 = Dropout(0.5)(dense_1)
dense_2 = Dense(64,activation= 'relu')(drop_1)
drop_2 = Dropout(0.5)(dense_2)
dense_3 = Dense(32,activation='relu')(drop_2)
out = Dense(2,activation = "softmax")(dense_3)
model = Model(inputs = input_layer,outputs = out)

In [107]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 768)]             0         
                                                                 
 dense (Dense)               (None, 524)               402956    
                                                                 
 dense_1 (Dense)             (None, 256)               134400    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8256  

In [109]:
model.compile(loss = "categorical_crossentropy",optimizer = "adam")
model.fit( X_train_pooled_output,train_y,epochs = 20,validation_data = (X_test_pooled_output,test_y),callbacks = [tensorboard_callback,cp_callback,score])

Epoch 1/20
668/676 [============================>.] - ETA: 0s - loss: 0.2240
Epoch 1: val_loss improved from -inf to 0.09964, saving model to training_1\cp.ckpt
290/290 [==============================] - 1s 2ms/step
val_f1_score 0.9697589481373265
676/676 [==============================] - 6s 7ms/step - loss: 0.2235 - val_loss: 0.0996
Epoch 2/20
675/676 [============================>.] - ETA: 0s - loss: 0.0978
Epoch 2: val_loss did not improve from 0.09964
290/290 [==============================] - 1s 3ms/step
val_f1_score 0.9694323144104804
676/676 [==============================] - 5s 8ms/step - loss: 0.0977 - val_loss: 0.0720
Epoch 3/20
669/676 [============================>.] - ETA: 0s - loss: 0.0607
Epoch 3: val_loss did not improve from 0.09964
290/290 [==============================] - 1s 2ms/step
val_f1_score 0.9872856298048492
676/676 [==============================] - 5s 8ms/step - loss: 0.0602 - val_loss: 0.0362
Epoch 4/20
675/676 [============================>.] - ETA: 0s -


##### -Observation :
###### As we can see from the above training we got best f1-score of 0.9936 which is very good from ML models i.e 0.99